##### ❇️ Azure ML: Deploying your model as a Web Service

In [ ]:
import pandas as pd
import sklearn
from sklearn.svm import SVC
import pickle
import joblib
from azureml.core import Workspace
from sklearn.model_selection import train_test_split

##### ❇️ Let's qickly train & save an irisclassifier model 
##### The model here is just a place holder you can train <br> and deploy model of your choice 

In [ ]:
# Download dataset
dataset = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
)
dataset.columns = ["Petal Length", "Petal Width", "Sepal Length", "Sepal Width", "Species"]

In [ ]:
dataset = dataset.replace(
    {"Species": {"Iris-setosa": 1, "Iris-versicolor": 2, "Iris-virginica": 3}}
)
dataset.head(5)

In [ ]:
# Train test split the data
X = dataset.drop(['Species'], axis=1)
y = dataset['Species']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [ ]:
# Instantiate a classifier
classifier = SVC(kernel = 'linear', random_state = 0)

In [ ]:
# Train classifier
classifier.fit(X_train, y_train)

In [ ]:
# Do prediction
y_pred = classifier.predict(X_test)

In [ ]:
## Save as a pickle file
filename= 'saved_model_v1.pkl'
joblib.dump(classifier,open(filename, 'wb'))

##### ❇️ Create a Workspace

In [ ]:
from azureml.core import Workspace
ws = Workspace.create(name='AzureML_Deployment_WS',
               subscription_id='2f##b8*****2',
               resource_group='AzureML_Deployment_RG',
               create_resource_group=True,
               location='eastus'
               )

##### ❇️ Register Model

In [ ]:
from azureml.core.model import Model

model = Model.register(ws, model_name="classifier", model_path="saved_model_v1.pkl")

##### ❇️ Setup Inference config

In [ ]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(
    conda_file='./env.yml',
    source_directory="./source_dir",
    entry_script="./score.py",
)

##### ❇️ Setup Deployment config

In [ ]:
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=2, memory_gb=3, auth_enabled=True
)

##### ❇️ Deploy the service

In [ ]:
from azureml.core.model import Model

service = Model.deploy(
    ws, # The instance of workspace created above
    "myservice",
    [Model(ws, 'bannerdetector')],
    inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)